In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Predict using saved models 

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.measure import compare_ssim as ssim
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras import backend as K

Import custom loss functions 

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
tf.config.experimental_run_functions_eagerly(True)
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(128,128,3))

def perceptual_loss(y_true, y_pred):
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
tf.config.experimental_run_functions_eagerly(True)
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(128,128,3))

def pixel_perceptual_loss(y_true, y_pred):
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    perceptual=K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))
    pixel= ((y_true-y_pred)**2)/(128*128*3)
    combined_loss= (0.5*pixel) + perceptual
    return (combined_loss)

Import demo datasets (on which model has not been trained)

In [0]:
pickle_in = open('/content/drive/My Drive/Demo/Demo10.pickle', 'rb')
X = pickle.load(pickle_in)
X_test = np.array(X)


In [0]:
pickle_in1 = open('/content/drive/My Drive/Demo/demo_original.pickle', 'rb')
y = pickle.load(pickle_in1)
y_test = np.array(y)

In [0]:
print (len(X_test))
print (len(y_test))


Choose model to train by uncommenting (ensure you use only associated dataset)

In [0]:
#autoencoder = load_model('/content/drive/My Drive/Trained_models/combined30.h5',custom_objects={'pixel_perceptual_loss':pixel_perceptual_loss})
#autoencoder = load_model('/content/drive/My Drive/Trained_models/perceptual30.h5',custom_objects={'perceptual_loss':perceptual_loss})
#autoencoder = load_model('/content/drive/My Drive/Trained_models/pixel_loss30.h5')

See prediction and evluation scores 

In [0]:
i = 11

original = y_test[i]
obfuscated = X_test[i]
predicted = autoencoder.predict(np.array([X_test[i]]))

plt.axis('off')
plt.imshow(obfuscated)
plt.show()

plt.axis('off')
plt.imshow(predicted[0])
plt.show()

print("Original")
plt.axis('off')
plt.imshow(original)
plt.show()


In [0]:
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
predicted = autoencoder.predict(np.array([X_test[i]]))
img=y_test[i]
img_x=X_test[i]
img_noise=predicted[0]
reconstructed_ssim = ssim(img, img_noise,
                  data_range=img_noise.max() - img_noise.min(),multichannel=True)#RECONSTRUCTED AND TRUE
obfuscated_ssim = ssim(img, img_x,
                  data_range=img_x.max() - img_x.min(),multichannel=True)#OBFUSCATED AND TRUE
print (i)
print (reconstructed_ssim)
print (obfuscated_ssim)

In [0]:
import numpy 
import math
import cv2
original = y_test[i]
predicted = autoencoder.predict(np.array([X_test[i]]))
contrast = predicted[0]#reconstructed
obfuscated= X_test[i]
#contrast = y_test[2]
#contrast = X_test[2]
def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
      return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

reconstructed_psnr= psnr(original,contrast)
obfuscated_psnr= psnr(original,obfuscated)

print (reconstructed_psnr)
print (obfuscated_psnr)